# Creating Robust Classes

In chapter three we coded a simple class in C++ using the syntax that we had introduced in that chapter. Of course, it was not possible (or even diserable) to discuss all possible 'syntax avenues' because doing so would be confusing. In this chapter we wish to create more robust and efficient classes and we realise this goal by using some of the syntax that C++ offers. In particular, we address a number of issues that have to do with data and object security, such as:

Issue 1: Ensuring that objects and their data are created in a safe way.

Issue 2: Accesing and using objects in a safe way; avoiding side-effects.

Issue 3: Working with object references rather than copies of objects.

Issue 4: Optimization: static objects and static member data.

This is quite a lot of territory to cover and the results in this chapter will be used again and again throughout this book. Thus, this chapter is a vital link to future chapters.

The most important topics are:
* Passing parameters to functions by value or by reference.
* Function overloading: ability to define several functions having the same name.
* More on constructors.
* Not all functions need be member functions: non-member functions.
* The `const` keyword and its consequences for C++ applications.

After having understood these topics and having implemented them in simple classes the reader will have reached a level of expertise approaching yellow belt. We discuss these topics not only because they are supported in C++ but because they help us become good C++ developers. They also promote the reliability and efficiency of our code.

# Call by reference and call by value
In C++ one can create functions taking zero or more arguments in their parameter list. To this end, we need discuss in what forms these arguments are created and used in a function. We take a simple example to motivate what we mean. Let us consider a function that calculates the larger of two numbers:

In [1]:
double Max(double x, double y)
{
    if (x > y)
        return x;
    return y;
}

This is a very simple function of course and in this case we say that the inputs parameters `x` and `y` are used in a *call-by-value* manner; this means that copies of these variables are made on the stack when the function is called:

In [2]:
#include <iostream>
{
    double d1 = 1.0;
    double d2 = -34536.00;
    
    // Copies of d1 and d2 offered to the function Max()
    double result = Max(d1, d2);
    std::cout << "Maxvalue is " << result << std::endl;
}

Maxvalue is 1


In this case we work with copies of `d1` and `d2` in the body of `Max()` and not `d1` and `d2` themselves. This process is taken care of automatically and you do not have to worry about this as programmer.

The call-by-value technique is also applicable, not only to built-in-types as we have just seen but also to class instances (objects). This means that objects (even `big` ones) will be copied if they are used in this call-by-value way. Let us take an example of a class having an embedded fixed-size array as member data:

In [1]:
#include <iostream>

class SampleClass
{
public: // For convenience only
    // This data created at compile time
    double contents[1000];

public:
    SampleClass(double d)
    {
        for (int i = 0; i < 1000; i++)
        {
            contents[i] = d;
        }
    }

    virtual ~SampleClass()
    {
        std::cout << "SampleClass instance being deleted\n";
    }
};

In [2]:
double Sum(SampleClass myClass)
{
    double result = myClass.contents[0];
    for (int i = 1; i < 1000; i++)
    {
        result += myClass.contents[i];
    }
    return result;
};

In [3]:
{
    SampleClass sc(1.0);
    double sum = Sum(sc);
}

SampleClass instance being deleted
SampleClass instance being deleted


In [4]:
double Sum2(SampleClass& myClass)
{
    double result = myClass.contents[0];
    for (int i = 1; i < 1000; i++)
    {
        result += myClass.contents[i];
    }
    return result;
};

In this case we work directly with the address of the input parameter and not a copy of it. We are, in fact working with the 'live' object and it is in fact possible to modify it in the body of the function that is being used in. For example, after having called the function `Sum2()` we could assign all of its values to zero! This is a side-effect and we shall see how to resolve this problem in the next section.

Some remarks on this section are:
* You can use call-by-value or call-by-reference for any type or class but in general I use the former for built-in types while I use the latter for objects and class instances.
* I try to avoid using pointers as input parameters to functions and we shall discuss why in a later chapter. By this statement we mean a function prototype having the following signature:

In [5]:
double Sum2(SampleClass* myClass);

This is the style reminiscent of bygone days and we advise against its use. We use modern C++ syntax and in this sense it is a 'better C'.
* When using call-by-reference it is only necessary to declare an object once as an address while in a main program you can use it as a 'normal' object (that is, without having to use `&`).
* An introduction to references, the stack and other topics relevant to the current context can be found in Chapter 22 and 23.

## Constant objects everywhere

One of the disadvantages of using references in argument lists is that the function is able to modify the input arguments. What we would ideally like to use is the address of an object while at the same time not be able to change the object in any way. To this end, we define the object to be a constant reference by introducing a new keyword as follows:

In [6]:
double Sum3(const SampleClass& myClass)
{
    return 2.0;// N.B. not possible to modify myClass
};

The body of this function can no longer modify the input argument and thus we avoid any side effects; the client code that delivered the object to the function can rest assured that the object will not be modified. In fact, if you try to modify the object you will receive a compiler error.

The above conclusions are valid for member functions as well as global functions and we shall give some examples in the coming section. But we first wish to discuss another aspect of 'constantness'.

### Read only (const) member functions
After having creating an object (by calling a constructor of its class) we would like to access its data by defining public member functions. To this end, we wish either to (a) give a copy of the data or (b) a reference to the data itself. We discuss the first case here and to this end we look at an example of a class that models two-dimensional points in Cartesian space. Having createad a point we would like to access its x and y coordinates in a read-only manner as it were:

In [2]:
!cat Point.hpp

// Point.hpp
//
// Header file for Points in two dimensions. A given Point has 3 coordinates
// for compatibility with other systems. However, it is not possible to
// influence the third coordinate and furthermore, the delivered functionality
// is typically two-dimensional.
//
// (C) Copyright Datasim BV 1995 - 2005

#ifndef Point_HPP
#define Point_HPP

#include <iostream>

class Shape
{
};

class Point : public Shape
{
private:
	double x;	// X coordinate
	double y;	// Y coordinate

	void init(double dx, double dy);

public:
	// Constructors
	Point();								// Default constructor
	Point(double xval, double yval);		// Initialize with x and y value
	Point(const Point& pt);					// Copy constructor
	~Point();								// Destructor

	// Accessing functions
	double X() const;					// The x-coordinate
	double Y() const;					// The y-coordinate

	// Modifiers
	void X(double NewX);				
	void Y(double NewY);

	// Arithmetic functions
	Point add(const Point& p) const;		// Return current + p
	Poi

In [3]:
!cat Point.cpp

// Point.cpp
//
// Header file for Points in two dimensions. A given Point has 3 coordinates
// for compatibility with other systems. However, it is not possible to
// influence the third coordinate and furthermore, the delivered functionality
// is typically two-dimensional.
//
// Last Modifications:
// chap04 AM Kick off
// chap05 AM Changed using const and reference
// chap05 AM Added copy constructor
// 2005-5-20 DD modified POINT -> Point
//
// (C) Copyright Datasim BV 1995 - 2006

#include "Point.hpp"

// Private functions
void Point::init(double dx, double dy)
{
	x = dx;
	y = dy;
}

Point::Point()
{// Default constructor
	init(0.0, 0.0);
}

Point::Point(double newx, double newy)
{// Initialize using newx and newy
	init(newx, newy);
}

/*
Point::Point(double newx, double newy) : x(newx), y(newy)
{// Initialize using newx and newy

	// init(newx, newy); NOT NEEDED
}
*/

Point::Point(const Point& pt)
{// Copy constructor
	x = pt.x;
	y = pt.y;
}

Point::~Point()
{
}

double Point::X

In [ ]:
!cat TestPoint.cpp

```cpp
class Point
{
private:
    void init(double xs, double ys);
    
    // Properties for x- and y-coordinates
    double x, y;
public:
    // Constructors and destructor
    Point(); // Default constructor
    Point(double xs, double ys); // Construct with coordinates
    Point(const Point& source); // Copy constructor
    virtual ~Point(); // Destructor
    
    // Selectors
    double X() const; // Return x
    double Y() const; // Return y
};
```

In [1]:
class Point
{
private:
    void init(double xs, double ys);

    // Properties for x- and y-coordinates
    double x, y;
public:
    // Constructors and destructor
    Point(); // Default constructor
    Point(double xs, double ys); // Construct with coordinates
    Point(const Point& source); // Copy constructor
    virtual ~Point(); // Destructor

    // Selectors
    double X() const; // Return x
    double Y() const; // Return y

    // Modifiers
	void X(double NewX);
	void Y(double NewY);
};

In [2]:
void Point::X(double NewX)
{
    x = NewX;
}
void Point::Y(double NewY)
{
    y = NewY;
}

input_line_8:7:1: error: function definition is not allowed here
{
^


Interpreter Error: 

Here we see that the functions `X()` and `Y()` are declared as being constant member functions and this means (in this case) that their bodies cannot modify the private member data of the point. This is like an insurance policy because client code can use these functions and be sure that its objects will not be modified. The body of the two selector functions above is given by:
```cpp
double Point::X() const
{// Return x
    return x;
}

double Point::Y() const
{// Return y
    return y;
}
```
In this case we state that copies of the member data are returned to the client, not the member data itself. This needs to be known and in other cases we may need to define functions that actually give the address of the member data as return types. This is an issue for a later chapter.

On the other hand, functions that modify the member data in general cannot be `const` for obvious reasons; they are, by definition functions that change the member data in some way. In this case the function prototypes are:
```cpp
// Modifiers
void X(double NewX); // Set x
void Y(double NewY); // Set y
```
while the body of these functions is given by:
```cpp
// Modifiers
void Point::X(double NewX)
{// Set x
    x = NewX;
}

void Point::Y(double NewY)
{// Set y
    y = NewY;
}
```
Here is an example:

In [ ]:
#include <iostream>
{
    Point p1(1.0, 3.14);
    // Read the coordinate on the Console
    std::cout << "First coordinate: " << p1.X() << std::endl;
    std::cout << "Second coordinate: " << p1.Y() << std::endl;
}